## Imports

In [ ]:
import os
os.environ["ISISROOT"] = "/usgs/pkgs/isis3.7.0/install"
os.environ["ISIS3DATA"] = "/usgs/cpkgs/isis3/data"
from pysis import isis

from plio.io import io_controlnetwork
from knoten.csm import create_csm
from scipy import sparse
import ale
import csmapi
import numpy as np

import matplotlib.pyplot as plt

from knoten.bundle import *

## Load Network and Generate Sensors

In [ ]:
cubes = 'data/cubes.lis'
sensors = generate_sensors(cubes)

network = 'data/hand_dense.net'
cnet = io_controlnetwork.from_isis(network)
cnet = compute_apriori_ground_points(cnet, sensors) # autoseed did not generate ground points, calculate and repopulate the data frame

## Determine Which Sensor Parameters to Solve For

In [ ]:
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()}
for sn, parameters in all_parameters.items():
    print(f"Image: {sn}")
    for param in parameters:
        print(f"  {param.name} | {param.index} | {param.value}")

In [ ]:
# Solve for angles and angular rates
solve_parameters = {sn: params[6:12] for sn, params in all_parameters.items()}

## Compute the Column Indices for Parameters

In [ ]:
column_dict = compute_coefficient_columns(cnet, sensors, solve_parameters)
# num_parameters = max(col_range[1] for col_range in column_dict.values())

## Compute the Weight Matrix
#### According to the weighted Normal equation (J.TWJ), W needs to be a square matrix the size of (# of measures)x2. So it is the weight of the observations. In ISIS, the weight of the observations are an inverted function of the size of the pixels on the focal plane (resolution). However, in csm we do not have access to that information. 
#### For the time being, since we are working exclusively with CTX images we are going to set the weight matrix equal to the identity matrix -> all observations have the same weight.

In [ ]:
num_observations = 2 * len(cnet)
W_observations = np.eye(num_observations) # this is a place holder until Jesse adds his calculations
W_params = compute_parameter_weights(cnet, sensors, solve_parameters, column_dict)

## Calculate Initial Sigma0

In [ ]:
def compute_sigma(V, dX, W_parameters, W_observations):
    """
    Computes the resulting standard deviation of the residuals for the current state of the bundle network.
    
    Parameters
    ----------
    V  :  np.array
          An array of residuals of the difference between registered measure 
          and back projected ground points in image space.
    W_parameters  :  ndarray 
                     The parameter weight matrix (i.e.: sensor parameters and point weights)
    W_observations  :  ndarray
                     The observation weight matrix (i.e.: point weights)
    
    Returns
    -------
       : float64
         Standard deviation of the residuals
    
    """
    num_parameters = W_parameters.shape[0]
    num_observations = W_observations.shape[0]
    dof = num_observations - num_parameters
    VTPV = V.dot(W_observations).dot(V) + dX.dot(W_parameters).dot(dX)
    sigma0 = np.sqrt(VTPV/dof)
    return sigma0

In [ ]:
V = compute_residuals(cnet, sensors)
dX = np.zeros(W_params.shape[0])
sigma0 = compute_sigma(V, dX, W_params, W_observations)

print((sigma0))

## Populate Jacobian

In [ ]:
J = compute_jacobian(cnet, sensors, solve_parameters, column_dict)

## Bundle Iteration

In [ ]:
def bundle_iteration(J, V, W_parameters, W_observations):
    """
    Parameters
    ----------
    J  :  ndarray
          The Jacobian matrix
    V  :  np.array
          An array of residuals of the difference between registered measure 
          and back projected ground points in image space.
    W_parameters  :  ndarray 
                     The parameter weight matrix (i.e.: sensor parameters and point weights)
    W_observations  :  ndarray
                     The observation weight matrix (i.e.: measure weights)
    
    Returns
    -------
    N  :  np.ndarray
          Normal equation matrix 
    
    dX  :  np.ndarray
           An array of updated parameter values
    """
    
    N = J.T.dot(W_observations).dot(J) + W_parameters
    C = J.T.dot(W_observations).dot(V)
    dX = np.linalg.inv(N).dot(C)
    return N, dX

In [ ]:
N, dX = bundle_iteration(J, V, W_params, W_observations)
print(dX.shape)

## Calculate Updated Sigma0

In [ ]:
dof = W_observations.shape[0] - W_params.shape[0]
updated_sigma0 = np.sqrt((V.dot(W_observations).dot(V) - dX.dot(J.T).dot(W_observations).dot(V))/dof)
print(updated_sigma0)

## Redundancy Number

In [ ]:
# redundancy for every measure
# vector will hold same order as the measures in the cnet df
# def compute_measure_redundancy
def compute_redundancy(N, W_observations, J):
    Qxx = np.linalg.inv(N)
    Qvv = np.linalg.inv(W_observations) - J.dot(Qxx).dot(J.T)
    r = np.diagonal(Qvv.dot(W_observations))
    
    return r

r = compute_redundancy(N, W_observations, J)
print(f'Minimum redundancy: {min(r)}')
print(f'Maximum redundancy: {max(r)}')
plt.boxplot(r)

## Update Sensors and Ground Points

In [ ]:
def update_parameters(sensors, parameters, network, updates, coefficient_columns):
    """
    Updates the sensor objects parameter values and the ground point values in the 
    networks DataFrame. The update occurs directly to variables, so nothing is returned.
    
    Parameters
    ----------
    sensors  :  dict
                A dictionary that maps ISIS serial numbers to CSM sensors
    parameters :  list
                  The list of  CsmParameter to compute the partials W.R.T.
    network  :  DataFrame
                The control network as a dataframe generated by plio.
    updates   : np.ndarray
                An array of updated parameter values
    coefficient_columns:  OrderedDict
                          Dictionary that maps serial numbers and point IDs to
                          the column range their parameters are in the Jacobian
                          matrix.
    
    Returns
    -------
    
    """
    # update the sensor partials
    for sn, sensor in sensors.items():
        for i, param in enumerate(parameters[sn]):
            if i > coefficient_columns[sn][1]:
                print('THIS SHOULD BE AN ACTUAL ERROR')
            updated_value = sensor.getParameterValue(param.index)
            updated_value += updates[coefficient_columns[sn][0]+i]
            sensor.setParameterValue(param.index, updated_value)

    # update ground points
    for _, row in network.iterrows():
        point_id = row['id']
        ground_pt = row[['adjustedX', 'adjustedY', 'adjustedZ']].values
        adj = updates[coefficient_columns[point_id][0]:coefficient_columns[point_id][1]] 
        network.loc[network.id == point_id, ["adjustedX", "adjustedY", "adjustedZ"]] = ground_pt + adj

## Whole bundle process in a loop

In [ ]:
sensors = generate_sensors(cubes) # generate sensors
cnet = io_controlnetwork.from_isis(network) # load in network
cnet = compute_apriori_ground_points(cnet, sensors) # calculate ground points

### INPUTS ###
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()} #all parameters
parameters = {sn: parameter[6:12] for sn, parameter in all_parameters.items()} #just solving for camera angles and angle velocity
##############

column_dict = compute_coefficient_columns(cnet, sensors, parameters)
num_parameters = max(col_range[1] for col_range in column_dict.values())
num_observations = 2 * len(cnet)
W_observations = np.eye(num_observations)
W_params = compute_parameter_weights(cnet, sensors, parameters, column_dict)

iteration = 0
V = compute_residuals(cnet, sensors)
dX = np.zeros(W_params.shape[0]) #initialize for sigma calculatio
sigma0 = compute_sigma(V, dX, W_params, W_observations)
print(f'iteration {iteration}: sigma0 = {sigma0}\n')

max_iterations = 30
tol = 1e-10
total_correction = np.zeros(num_parameters)
for i in range(max_iterations):   
    iteration += 1
    old_sigma0 = sigma0
    
    J = compute_jacobian(cnet, sensors, parameters, column_dict)    
    N = J.T.dot(W_observations).dot(J) + W_params # calculate the normal equation
    C = J.T.dot(W_observations).dot(V) - W_params.dot(total_correction)
    dX = np.linalg.inv(N).dot(C) #calculate change in camera parameters and ground points
    total_correction += dX
    print(f'corrections: mean = {dX.mean()} min = {dX.min()} max = {dX.max()}')
    
    update_parameters(sensors, parameters, cnet, dX, column_dict)
    
    V = compute_residuals(cnet, sensors)
    sigma0 = compute_sigma(V, dX, W_params, W_observations)
    print(f'iteration {iteration}: sigma0 = {sigma0}\n')
    
    if (abs(sigma0 - old_sigma0) < tol):
        print(f'change in sigma0 of {abs(sigma0 - old_sigma0)} converged!')
        break
    